<a href="https://colab.research.google.com/github/Thiagorcj/NLP_projects/blob/main/Naive_bayes_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercício 2 de PLN - Análise de Sentimentos

### Professor: Yuri Malheiros
### Aluno: Thiago Rodrigues Cruz Justino

### Descrição:
O objetivo deste exercício é treinar e avaliar um classificador Naive Bayes para
uma tarefa de análise de sentimentos.
O texto utilizado no treinamento e no teste deve ser pré-processado para incluir
o prefixo “NAO_” de acordo com a seguinte regra:

O prefixo “NAO_” deve ser adicionado a cada palavra após um
token de negação (não, nem, nunca, jamais, tampouco) até a próxima
pontuação. São consideradas pontuações: . , ? ! ;

O treino deve ser realizado com 80% dos dados e o teste com 20%. Você deve
calcular a acurácia do classificador aplicado aos dados de teste e comparar com
o resultado do Naive Bayes sem a utilização do prefixo “NAO_”.


### Dados:
Os dados utilizados são os dados do corpus de reviews de produtos da B2W
disponibilizado em: https://github.com/yurimalheiros/pln/tree/main/datasets

### Bibliotecas usadas:

In [55]:
import re
import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Obtendo o datataframe:

In [2]:
df = pd.read_csv('B2W-Reviews01.csv')
df = df.dropna(subset=['review_text']) # Retirando as linhas com valores nulos
df['label'] = df['overall_rating'].apply(lambda x: 0 if x <=3 else 1) # Gerando um nova classificação binária, 0 se a avaliação for menor que 3 e 1 se for maior igual.
df.head(5)

<ipython-input-2-f4db5678d25d>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('B2W-Reviews01.csv')
<ipython-input-2-f4db5678d25d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['overall_rating'].apply(lambda x: 0 if x <=3 else 1) # Gerando um nova classificação binária, 0 se a avaliação for menor que 3 e 1 se for maior igual.


,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,label
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ,1
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC,1
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP,1
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP,1
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG,1


### Testes e calculando a acurácia:

In [10]:
X = df['review_text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #Dividindo os dados em 80% de treino e 20% de teste
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test) # Obs: Lembrar que não se deve utilizar fit nos dados de teste
clf = MultinomialNB()
model = clf.fit(X_train_vec, y_train)

print(f'Acurácia sem utilizar _NOT → {accuracy_score(y_test, model.predict(X_test_vec)):.4f}')

Acurácia sem utilizar _NOT → 0.8605


### Solução do exercício:
Função ``replace_func``

Esta função é usada para modificar o texto de acordo com a regra específica de adição do prefixo _NOT. Ela pega os grupos de negação, texto e pontuação,respectivamente, e adiciona o prefixo antes de cada palavra do grupo texto.

Função ``add_prefix``

Esta função pega o texto, o agrupa em negação, texto e pontuação e aplica na função ``replace_func`` retornando o texto alterado.

In [29]:
# Função auxiliar para substituir o texto
def replace_func(match):
    negation = match.group(1)
    text_to_modify = match.group(2)
    punctuation = match.group(3)
    modified_text = re.sub(r'\b(\w+)\b', r'NAO_\1', text_to_modify)
    return f'{negation} {modified_text}{punctuation}'

# Função para adicionar o prefixo "NAO_"
def add_prefix(text):
    # Definindo o padrão para tokens de negação e pontuação
    pattern = re.compile(r'\b(não|nem|nunca|jamais|tampouco)\b(.*?)([.,?!;])', re.IGNORECASE)
    # Substituindo o texto
    return pattern.sub(replace_func, text)

# Adicionando o prefixo "NAO_" ao texto
df['text_with_prefix'] = df['review_text'].apply(add_prefix)

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df['text_with_prefix'], df['label'], test_size=0.2, random_state=42)

# Vetorização dos dados
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Treinando o modelo com prefixo
model_with_prefix = MultinomialNB()
model_with_prefix.fit(X_train_vec, y_train)

# Fazendo previsões e calcular a acurácia
y_pred_with_prefix = model_with_prefix.predict(X_test_vec)
accuracy_with_prefix = accuracy_score(y_test, y_pred_with_prefix)

# Treinando o modelo sem o prefixo
X_train_no_prefix, X_test_no_prefix, y_train_no_prefix, y_test_no_prefix = train_test_split(df['review_text'], df['label'], test_size=0.2, random_state=42)
X_train_vec_no_prefix = vectorizer.fit_transform(X_train_no_prefix)
X_test_vec_no_prefix = vectorizer.transform(X_test_no_prefix)

model_no_prefix = MultinomialNB()
model_no_prefix.fit(X_train_vec_no_prefix, y_train_no_prefix)

# Fazendo previsões e calcular a acurácia
y_pred_no_prefix = model_no_prefix.predict(X_test_vec_no_prefix)
accuracy_no_prefix = accuracy_score(y_test_no_prefix, y_pred_no_prefix)

# Mostrando os resultados
print(f'Acurácia com prefixo "NAO_" → {accuracy_with_prefix:.4f}')
print(f'Acurácia sem prefixo "NAO_" → {accuracy_no_prefix:.4f}')

Acurácia com prefixo "NAO_" → 0.8562
Acurácia sem prefixo "NAO_" → 0.8560


## Extra: Implementando o Naive Bayes
Abaixo está uma implementação do Naive Bayes, com objetivo de revisar esse metódo de classificação e os conceitos ligados. Percebe-se que o tempo de execução é bem maior que o normal, isso se dá, provavelmente, porque não há a vetorização das palavras, escolhi fazer sem vetorizar para poder visualizar as palavras e suas contagens por meio da função ``get_top_words_per_category``.

In [56]:
class NaiveBayesClassifier:
    def __init__(self):
        self.stopwords = set(stopwords.words('portuguese'))
        self.contagem_palavras_por_categoria = defaultdict(lambda: defaultdict(int))
        self.num_palavras_por_categoria = defaultdict(int)

    def tokenize(self, x):
        texto = re.sub(r'\d+', '', x)  # Removendo números
        texto = re.sub(r'[^\w\s]', '', texto)  # Removendo pontuação
        tokens = word_tokenize(texto, language='portuguese')
        return [word for word in tokens if word not in self.stopwords]  # Removendo stopwords

    def train(self,label_column,texts_column, df_filtrado):
        for idx, row in df_filtrado.iterrows():
            categoria = row[label_column]
            lista_palavras = self.tokenize(row[texts_column])
            for palavra in lista_palavras:
                self.contagem_palavras_por_categoria[categoria][palavra] += 1
                self.num_palavras_por_categoria[categoria] += 1

    def predict(self, x):
        x = self.tokenize(x)
        proporcoes_por_categoria = {}

        # Calculando a probabilidade logarítmica para cada categoria
        for categoria in self.contagem_palavras_por_categoria.keys():
            log_prob_categoria = np.log(self.num_palavras_por_categoria[categoria] / sum(self.num_palavras_por_categoria.values()))  # Log da probabilidade da categoria
            log_prob_palavras = 0.0  # Inicializando o log da probabilidade das palavras

            for palavra in x:
                if palavra in self.contagem_palavras_por_categoria[categoria]:
                    # Log da probabilidade da palavra dado que é da categoria
                    log_prob_palavras += np.log(self.contagem_palavras_por_categoria[categoria][palavra] / self.num_palavras_por_categoria[categoria])
                else:
                    # Caso a palavra não esteja na categoria, usa um valor pequeno para a probabilidade
                    log_prob_palavras += np.log(1 / (self.num_palavras_por_categoria[categoria] + len(self.contagem_palavras_por_categoria[categoria])))  # Utilizando Laplace smoothing

            proporcoes_por_categoria[categoria] = log_prob_categoria + log_prob_palavras

        # Retornando a categoria com a maior probabilidade
        return  max(proporcoes_por_categoria, key=proporcoes_por_categoria.get)

    def get_top_words_per_category(self, top_n=10):
        top_words = {}
        for categoria, palavras in self.contagem_palavras_por_categoria.items():
            # Ordenando as palavras pela frequência e seleciona as top_n palavras
            sorted_words = sorted(palavras.items(), key=lambda x: x[1], reverse=True)
            top_words[categoria] = sorted_words[:top_n]
        return top_words

# Dividindo o DataFrame em conjuntos de treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Treinamento do classificador
nb = NaiveBayesClassifier()
nb.train('label', 'review_text', train_df)

# Predição e cálculo da acurácia
y_true = test_df['label']
y_pred = test_df['review_text'].apply(lambda x: nb.predict(x))

# Cálculo da acurácia
accuracy = np.mean(y_true == y_pred)
print(f'Acurácia: {accuracy:.4f}')

Acurácia: 0.8541


In [54]:
# Obtendo palavras mais frequentes por categoria
top_words = nb.get_top_words_per_category(top_n=5)
print("Palavras mais frequentes por categoria:")
for categoria, palavras in top_words.items():
    print(f"{categoria}:")
    for palavra, freq in palavras:
        print(f"  {palavra}: {freq}")

Palavras mais frequentes por categoria:
1:
  produto: 22097
  bom: 12910
  entrega: 9903
  bem: 9344
  qualidade: 9149
0:
  produto: 20475
  O: 8246
  Não: 6671
  A: 5246
  recebi: 4986
